# Telecom Customer Data Cleaning

This notebook covers step-by-step data cleaning for a telecom customer call records dataset.

---

## 1. Load Dataset and Initial Inspection

In [2]:
import pandas as pd

# Load the dataset
df = pd.read_csv('C:\\Users\\Ali\\Desktop\\Call Details-Data.csv')

# Show initial shape and info
print(f"Initial dataset shape: {df.shape}")

Initial dataset shape: (101174, 17)


In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 101174 entries, 0 to 101173
Data columns (total 17 columns):
 #   Column          Non-Null Count   Dtype  
---  ------          --------------   -----  
 0   Phone Number    101174 non-null  object 
 1   Account Length  101174 non-null  int64  
 2   VMail Message   101174 non-null  int64  
 3   Day Mins        101174 non-null  float64
 4   Day Calls       101174 non-null  int64  
 5   Day Charge      101174 non-null  float64
 6   Eve Mins        101174 non-null  float64
 7   Eve Calls       101174 non-null  int64  
 8   Eve Charge      101174 non-null  float64
 9   Night Mins      101174 non-null  float64
 10  Night Calls     101174 non-null  int64  
 11  Night Charge    101174 non-null  float64
 12  Intl Mins       101174 non-null  float64
 13  Intl Calls      101174 non-null  int64  
 14  Intl Charge     101174 non-null  float64
 15  CustServ Calls  101174 non-null  int64  
 16  Churn           101174 non-null  bool   
dtypes: bool(1)

In [4]:
df.head()

,Phone Number,Account Length,VMail Message,Day Mins,Day Calls,Day Charge,Eve Mins,Eve Calls,Eve Charge,Night Mins,Night Calls,Night Charge,Intl Mins,Intl Calls,Intl Charge,CustServ Calls,Churn
0,382-4657,128,25,265.1,110,45.07,197.4,99,16.78,244.7,91,11.01,10.0,3,2.70,1,False
1,371-7191,107,26,161.6,123,27.47,195.5,103,16.62,254.4,103,11.45,13.7,3,3.70,1,False
2,358-1921,137,0,243.4,114,41.38,121.2,110,10.30,162.6,104,7.32,12.2,5,3.29,0,False
3,375-9999,84,0,299.4,71,50.90,61.9,88,5.26,196.9,89,8.86,6.6,7,1.78,2,False
4,330-6626,75,0,166.7,113,28.34,148.3,122,12.61,186.9,121,8.41,10.1,3,2.73,3,False


## 2. Check Missing Values

In [5]:
# Count missing values per column
missing = df.isnull().sum()
print("Missing values per column:")
print(missing)

Missing values per column:
Phone Number      0
Account Length    0
VMail Message     0
Day Mins          0
Day Calls         0
Day Charge        0
Eve Mins          0
Eve Calls         0
Eve Charge        0
Night Mins        0
Night Calls       0
Night Charge      0
Intl Mins         0
Intl Calls        0
Intl Charge       0
CustServ Calls    0
Churn             0
dtype: int64


## 3. Remove Duplicate Rows

In [7]:
dup_count = df.duplicated().sum()
print(f"Duplicate rows before removal: {dup_count}")

Duplicate rows before removal: 40729


In [12]:
df = df.drop_duplicates()

In [13]:
dup_count_after = df.duplicated().sum()
print(f"Duplicate rows after removal: {dup_count_after}")

Duplicate rows after removal: 0


## 4. Clean Phone Numbers

- Remove spaces, dashes, parentheses
- Ensure country code prefix (e.g., +1) exists

In [19]:
import re
import pandas as pd

def clean_phone(phone):
    if pd.isna(phone):
        return phone
    phone = str(phone).strip()
    # Remove all characters except digits and plus sign
    phone = re.sub(r'[^\d+]', '', phone)
    # Add +1 country code if missing (change this as needed)
    if not phone.startswith('+'):
        phone = '+1' + phone
    return phone

# Assuming your phone number column is named 'Phone Number'
df['Phone Number'] = df['Phone Number'].apply(clean_phone)

# Show first few cleaned phone numbers
print(df['Phone Number'].head())


0    +13824657
1    +13717191
2    +13581921
3    +13759999
4    +13306626
Name: Phone Number, dtype: object


#### 6. Detect and Remove Outliers (Numeric Columns)

- Using IQR method
- Visualize with boxplots

In [20]:
import matplotlib.pyplot as plt
import seaborn as sns

def detect_outliers_iqr(data, col):
    Q1 = data[col].quantile(0.25)
    Q3 = data[col].quantile(0.75)
    IQR = Q3 - Q1
    lower_bound = Q1 - 1.5*IQR
    upper_bound = Q3 + 1.5*IQR
    outliers = data[(data[col] < lower_bound) | (data[col] > upper_bound)]
    return outliers.index

numeric_cols = ['call_duration', 'charge_amount']  # change as per dataset

for col in numeric_cols:
    if col in df.columns:
        plt.figure(figsize=(8,4))
        sns.boxplot(x=df[col])
        plt.title(f'Boxplot for {col}')
        plt.show()

        outlier_idxs = detect_outliers_iqr(df, col)
        print(f'Number of outliers detected in {col}: {len(outlier_idxs)}')

        # Drop outliers
        df = df.drop(index=outlier_idxs)

print(f"Dataset shape after outlier removal: {df.shape}")

Dataset shape after outlier removal: (60445, 17)


## 7. Final Check and Save Cleaned Data

In [21]:
print("Final dataset info:")
df.info()

Final dataset info:
<class 'pandas.core.frame.DataFrame'>
Index: 60445 entries, 0 to 101173
Data columns (total 17 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Phone Number    60445 non-null  object 
 1   Account Length  60445 non-null  int64  
 2   VMail Message   60445 non-null  int64  
 3   Day Mins        60445 non-null  float64
 4   Day Calls       60445 non-null  int64  
 5   Day Charge      60445 non-null  float64
 6   Eve Mins        60445 non-null  float64
 7   Eve Calls       60445 non-null  int64  
 8   Eve Charge      60445 non-null  float64
 9   Night Mins      60445 non-null  float64
 10  Night Calls     60445 non-null  int64  
 11  Night Charge    60445 non-null  float64
 12  Intl Mins       60445 non-null  float64
 13  Intl Calls      60445 non-null  int64  
 14  Intl Charge     60445 non-null  float64
 15  CustServ Calls  60445 non-null  int64  
 16  Churn           60445 non-null  bool   
dtypes: bool(1), flo

In [24]:
df.to_csv('C:\\Users\\Ali\\Desktop\\cleaned_cdr.csv', index=False)
print("Cleaned data saved to C:\\Users\\Ali\\Desktop\\cleaned_cdr.csv")

Cleaned data saved to C:\Users\Ali\Desktop\cleaned_cdr.csv
